In [4]:
import datetime

now = datetime.datetime.now()
print("Current date and time:", now)


Current date and time: 2025-06-30 17:10:47.253490


In [3]:
import pandas as pd
from tabulate import tabulate

# Load the results CSV
csv_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\results.csv"
df = pd.read_csv(csv_path)

# Map short model names to full names
model_name_map = {
    "RF": "RF - Random Forest",
    "GBRT": "GBRT - Gradient Boosting",
    "XGB": "XGB - XGBoost",
    "LR": "LR - Linear Regression",
    "PR": "PR - Polynomial Regression"
}
df["Model"] = df["Model"].map(model_name_map)

# Rule 1: Filter models with small R² difference
threshold = 0.5
df_filtered = df[abs(df["R2_Test"] - df["R2_Train"]) <= threshold].copy()

best_models_rule1 = []
best_models_rule2 = []

for river in df["River_Name"].unique():
    df_river_filtered = df_filtered[df_filtered["River_Name"] == river]

    if not df_river_filtered.empty:
        best_model = df_river_filtered.loc[df_river_filtered["MAPE_Test"].idxmin()].copy()
        best_model["Selection_Rule"] = "Rule 1: Stable R²"
        best_models_rule1.append(best_model)
    else:
        df_river_full = df[df["River_Name"] == river]
        best_model = df_river_full.loc[df_river_full["MAPE_Test"].idxmin()].copy()
        best_model["Selection_Rule"] = "Rule 2: Fallback by MAPE"
        best_models_rule2.append(best_model)

# Combine both rule groups
all_best_models = pd.DataFrame(best_models_rule1 + best_models_rule2)

# Sort for clarity
all_best_models = all_best_models.sort_values(by=["Selection_Rule", "System"])

# Columns to display
columns_to_show = [
    "Model", "River_Name", "R2_Test", "MSE_Test", "MAPE_Test",
    "R2_Train", "MSE_Train", "MAPE_Train",
    "System", "TopK_Features", "Additional_Features_Used", "ARIMA_Enabled",
    "Feature_Names", "Selection_Rule"
]

best_models_formatted = all_best_models[columns_to_show].copy()

# Round numeric values
float_cols = ["R2_Test", "MSE_Test", "MAPE_Test", "R2_Train", "MSE_Train", "MAPE_Train"]
best_models_formatted[float_cols] = best_models_formatted[float_cols].applymap(lambda x: round(x, 4))

# Separate by rule and print
df_rule1 = best_models_formatted[best_models_formatted["Selection_Rule"] == "Rule 1: Stable R²"]
df_rule2 = best_models_formatted[best_models_formatted["Selection_Rule"] == "Rule 2: Fallback by MAPE"]

print("✅ Best Models per River (Rule 1: Stable R²)\n")
print(tabulate(df_rule1, headers="keys", tablefmt="github", showindex=False))

print("\n✅ Best Models per River (Rule 2: Fallback by MAPE)\n")
print(tabulate(df_rule2, headers="keys", tablefmt="github", showindex=False))

# Save best models to CSV
output_path = r"C:\Users\MuriloFarias\Desktop\NNS-JULIA\PredictSalmonRuns\murilo_salmon\julia_models\best_models_per_river.csv"
best_models_formatted.to_csv(output_path, index=False)
print(f"\n✅ Best models saved to:\n{output_path}")



✅ Best Models per River (Rule 1: Stable R²)

| Model                      | River_Name            |   R2_Test |    MSE_Test |   MAPE_Test |   R2_Train |   MSE_Train |   MAPE_Train | System         |   TopK_Features | Additional_Features_Used   | ARIMA_Enabled   | Feature_Names                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | Selection_Rule    |
|----------------------------|-----------------------|-----------|-------------|-------------|------------|-------------|--------------|----------------|-----------------|--------------------

C:\Users\MuriloFarias\AppData\Local\Temp\ipykernel_24940\1688200386.py:56: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_models_formatted[float_cols] = best_models_formatted[float_cols].applymap(lambda x: round(x, 4))
